# Udacity Machine Learning Nanodegree
--------------


## Capstone Project on Vehicle-Detection
---------------


### Parameter Tuning for SVM model

#### Imports

In [6]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time 
from sklearn.svm import LinearSVC

In [7]:
import pickle
%matplotlib inline

--------

### Classifier

#### Data

In [21]:
cars = glob.glob('data/vehicles/**/*.png')
notcars = glob.glob('data/non-vehicles/**/*.png')

In [22]:
# Reduce the sample size because HOG features are slow to compute
sample_size = 1000
cars = cars[0:sample_size]
notcars = notcars[0:sample_size]

#### Parameters

In [23]:
colorspace = 'YCrCb'  #'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
hist_bins = 32
hist_bins_range = (0,256)
spatial_size = (32,32)

#### Extraction of all required features

In [24]:
from project.feature_extraction import FeatureExtraction

In [25]:
t=time.time()

FE = FeatureExtraction(hist_bins = hist_bins,\
                            hist_bins_range =  hist_bins_range,\
                            spatial_size = spatial_size,\
                            orient = orient,\
                            pix_per_cell =  pix_per_cell,\
                            cell_per_block =  cell_per_block)


color_car_features = FE.extract_color_features(cars,
                                               cspace=colorspace)
color_notcar_features = FE.extract_color_features(notcars,
                                                cspace=colorspace)

hog_car_features = FE.extract_hog_features(cars,
                                    cspace=colorspace,
                                    hog_channel=hog_channel)

hog_notcar_features = FE.extract_hog_features(notcars,
                                       cspace=colorspace,
                                       hog_channel=hog_channel)

t2 = time.time()

print(round(t2-t, 2), 'Seconds to extract HOG features and color features...')


/Users/dibakarsigdel/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


14.01 Seconds to extract HOG features and color features...


-------

#### Stacking all feature to single vector

In [26]:
# Create an array stack of feature vectors
X_car = np.hstack((color_car_features,hog_car_features)).astype(np.float64)
X_notcar = np.hstack((color_notcar_features, hog_notcar_features)).astype(np.float64) 
X = np.vstack((X_car,X_notcar)).astype(np.float64) 

X_scalar = StandardScaler().fit(X)

# Apply the scaler to X
scaled_X = X_scalar.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(hog_car_features)), np.zeros(len(hog_notcar_features))))

#### Train and test split

In [27]:
from sklearn.utils import shuffle

scaled_X, y = shuffle(scaled_X, y )

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 6156


#### Parameter tuning

In [29]:
from sklearn import grid_search,svm
from sklearn.model_selection import GridSearchCV

In [30]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [31]:
best = svc_param_selection(scaled_X, y,4)

In [32]:
best

{'C': 10, 'gamma': 0.001}

#### Getting Best model

In [33]:
# Use a linear SVC 
svc = svm.SVC(kernel='rbf',C =10,gamma = 0.001)
# Check the training time for the SVC


t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()


n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

14.01 Seconds to train SVC...
Test Accuracy of SVC =  0.835
My SVC predicts:  [ 1.  0.  0.  0.  0.  0.  1.  0.  0.  1.]
For these 10 labels:  [ 1.  0.  1.  1.  0.  0.  1.  0.  0.  1.]
0.08806 Seconds to predict 10 labels with SVC


In [34]:
import pickle as pickle
dist_pickle = {}
dist_pickle["svc"] = svc 
dist_pickle["scalar"] = X_scalar
dist_pickle["orient"] = orient 
dist_pickle["pix_per_cell"]= pix_per_cell 
dist_pickle["cell_per_block"]= cell_per_block 
dist_pickle["spatial_size"]= spatial_size 
dist_pickle["hist_bins"]= hist_bins 

with open('svc_rbf_pickle.p', mode='wb') as f:
    pickle.dump(dist_pickle, f)

--------

#### Training Linear SVC

In [26]:
# Use a linear SVC 
svc = LinearSVC(C =0.001)
# Check the training time for the SVC


t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()


n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

392.39 Seconds to train SVC...
Test Accuracy of SVC =  0.9904
My SVC predicts:  [ 1.  0.  0.  0.  0.  1.  0.  0.  1.  0.]
For these 10 labels:  [ 1.  0.  0.  0.  0.  1.  0.  0.  1.  0.]
2.42908 Seconds to predict 10 labels with SVC


#### Saving model for later use

In [27]:
import pickle as pickle
dist_pickle = {}
dist_pickle["svc"] = svc 
dist_pickle["scaler"] = X_scaler
dist_pickle["orient"] = orient 
dist_pickle["pix_per_cell"]= pix_per_cell 
dist_pickle["cell_per_block"]= cell_per_block 
dist_pickle["spatial_size"]= spatial_size 
dist_pickle["hist_bins"]= hist_bins 

with open('svc_pickle.p', mode='wb') as f:
    pickle.dump(dist_pickle, f)

------